In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from config import api_key

In [17]:
base_url = "https://yfapi.net/v6/finance/quote"
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'FB', 'GOOG', 'TSLA', 'NVDA', 'JPM', 'UNH']

In [20]:
stock_data = []
for ticker in tickers:
    stock_dict = {}
    try:
        querystring = {"symbols": ticker}

        headers = {
            'x-api-key': api_key
            }
        
        response = requests.get(base_url, headers=headers, params=querystring).json()
        symbol = response['quoteResponse']['result'][0]['symbol']
        day_open = response['quoteResponse']['result'][0]['regularMarketOpen']
        day_high = response['quoteResponse']['result'][0]['regularMarketDayHigh']
        day_low = response['quoteResponse']['result'][0]['regularMarketDayLow']
        rating = response['quoteResponse']['result'][0]['averageAnalystRating']
        
        stock_dict['ticker'] = symbol
        stock_dict['day_open'] = day_open
        stock_dict['day_high'] = day_high
        stock_dict['day_low'] = day_low
        stock_dict['avg_analyst_rating'] = rating
        
        stock_data.append(stock_dict)
        
        print('---------')
        print(f'{symbol} Rating: {rating}')
        print(f'{symbol} Open: {day_open}; High: {day_high}; Low: {day_low}')
    
    except:
        print('---------')
        print(f'{ticker} Missing Data')

---------
AAPL Rating: 1.8 - Buy
AAPL Open: 163.51; High: 166.35; Low: 163.015
---------
GOOGL Rating: 1.7 - Buy
GOOGL Open: 2723.27; High: 2739.9988; Low: 2681.845
---------
MSFT Rating: 1.7 - Buy
MSFT Open: 298.89; High: 300.14; Low: 294.9
---------
AMZN Rating: 1.7 - Buy
AMZN Open: 3222.415; High: 3261.68; Low: 3191.06
---------
FB Rating: 2.1 - Buy
FB Open: 214.5; High: 214.71; Low: 207.63
---------
GOOG Rating: 1.5 - Strong Buy
GOOG Open: 2736.95; High: 2750.22; Low: 2692.23
---------
TSLA Rating: 2.6 - Hold
TSLA Open: 914.98; High: 942.8; Low: 907.09
---------
NVDA Rating: 2.0 - Buy
NVDA Open: 265.07; High: 271.5199; Low: 259.67
---------
JPM Rating: 2.5 - Buy
JPM Open: 140.35; High: 140.76; Low: 138.7301
---------
UNH Rating: 1.9 - Buy
UNH Open: 505.01; High: 513.0; Low: 504.36


In [21]:
stock_data

[{'ticker': 'AAPL',
  'day_open': 163.51,
  'day_high': 166.35,
  'day_low': 163.015,
  'avg_analyst_rating': '1.8 - Buy'},
 {'ticker': 'GOOGL',
  'day_open': 2723.27,
  'day_high': 2739.9988,
  'day_low': 2681.845,
  'avg_analyst_rating': '1.7 - Buy'},
 {'ticker': 'MSFT',
  'day_open': 298.89,
  'day_high': 300.14,
  'day_low': 294.9,
  'avg_analyst_rating': '1.7 - Buy'},
 {'ticker': 'AMZN',
  'day_open': 3222.415,
  'day_high': 3261.68,
  'day_low': 3191.06,
  'avg_analyst_rating': '1.7 - Buy'},
 {'ticker': 'FB',
  'day_open': 214.5,
  'day_high': 214.71,
  'day_low': 207.63,
  'avg_analyst_rating': '2.1 - Buy'},
 {'ticker': 'GOOG',
  'day_open': 2736.95,
  'day_high': 2750.22,
  'day_low': 2692.23,
  'avg_analyst_rating': '1.5 - Strong Buy'},
 {'ticker': 'TSLA',
  'day_open': 914.98,
  'day_high': 942.8,
  'day_low': 907.09,
  'avg_analyst_rating': '2.6 - Hold'},
 {'ticker': 'NVDA',
  'day_open': 265.07,
  'day_high': 271.5199,
  'day_low': 259.67,
  'avg_analyst_rating': '2.0 - Buy

In [23]:
url = 'https://finance.yahoo.com/'
# Scraping for latest news article on each ticker
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been s

In [38]:
# Scraping for latest news article on each ticker
stock_articles = []
for ticker in tickers:
    articles_dict = {}
    
    url = f'https://finance.yahoo.com/quote/{ticker}/news?p={ticker}'
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    latest_article = soup.find('a', class_='js-content-viewer')
    article_title = latest_article.text
    article_link = latest_article['href']
    
    articles_dict['ticker'] = ticker
    articles_dict['article_title'] = article_title
    articles_dict['link'] = f'https://finance.yahoo.com/{article_link}'
    
    stock_articles.append(articles_dict)
    
    print('----------')
    print(ticker)
    print(article_title)
    print(f'https://finance.yahoo.com/{article_link}')

----------
AAPL
Apple Resolves Outage That Hobbled Apps and Internal Systems
https://finance.yahoo.com//news/apple-suffers-widespread-outage-hitting-170314183.html
----------
GOOGL
Sony Entertainment to Buy Jade Raymond’s Haven Gaming Studio
https://finance.yahoo.com//news/sony-entertainment-buy-jade-raymond-162114298.html
----------
MSFT
Who will be the first trillionaire? Study predicts Elon Musk in 2024.
https://finance.yahoo.com//news/who-will-be-the-first-trillionaire-study-predicts-elon-musk-in-2024-144150081.html
----------
AMZN
Vinco Ventures stock soars, Goldman Sachs conducts first OTC bitcoin option trade, Al Michaels strikes deal with Amazon
https://finance.yahoo.com//video/vinco-ventures-stock-soars-goldman-195710086.html
----------
FB
Market check: Stocks fall as oil extends gains
https://finance.yahoo.com//video/market-check-stocks-fall-oil-192426504.html
----------
GOOG
Sony Entertainment to Buy Jade Raymond’s Haven Gaming Studio
https://finance.yahoo.com//news/sony-ent

In [39]:
stock_articles

[{'ticker': 'AAPL',
  'article_title': 'Apple Resolves Outage That Hobbled Apps and Internal Systems',
  'link': 'https://finance.yahoo.com//news/apple-suffers-widespread-outage-hitting-170314183.html'},
 {'ticker': 'GOOGL',
  'article_title': 'Sony Entertainment to Buy Jade Raymond’s Haven Gaming Studio',
  'link': 'https://finance.yahoo.com//news/sony-entertainment-buy-jade-raymond-162114298.html'},
 {'ticker': 'MSFT',
  'article_title': 'Who will be the first trillionaire? Study predicts Elon Musk in 2024.',
  'link': 'https://finance.yahoo.com//news/who-will-be-the-first-trillionaire-study-predicts-elon-musk-in-2024-144150081.html'},
 {'ticker': 'AMZN',
  'article_title': 'Vinco Ventures stock soars, Goldman Sachs conducts first OTC bitcoin option trade, Al Michaels strikes deal with Amazon',
  'link': 'https://finance.yahoo.com//video/vinco-ventures-stock-soars-goldman-195710086.html'},
 {'ticker': 'FB',
  'article_title': 'Market check: Stocks fall as oil extends gains',
  'link'

In [40]:
browser.quit()